# 4th attempt - CNN

In [1]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
SIZE = 5
AMOUNT_BOARDS = 10000

In [3]:
gen = 2
name_df = f'{PATH_DF}\\{SIZE}-{AMOUNT_BOARDS}\\{SIZE}size_{AMOUNT_BOARDS}boards_{gen}gen_reverse'
reverse_df = pd.read_pickle(f'{name_df}.pkl')

In [4]:
new_columns = [f'Col_{i}' for i in range(gen*SIZE*SIZE)]
reverse_df_sort = reverse_df.sort_values(by = new_columns).reset_index(drop=True)
for i in reverse_df_sort.columns:
    reverse_df_sort[i] = reverse_df_sort[i].astype(int)

In [5]:
print("reverse df:", len(reverse_df))
print("reverse df sort:",len(reverse_df_sort))

reverse df: 29760
reverse df sort: 29760


In [6]:
# Step 1: Prepare Data
amount_features = len(reverse_df_sort.columns) - SIZE*SIZE #the previous boards
features = reverse_df_sort.iloc[:, :amount_features]
name_col = 'Col_' + str(amount_features + 1)  # Target: the first pixel in the board
target = reverse_df_sort[name_col]

# Step 2: Split Data
X_train_val, X_test, y_train_val, y_test = train_test_split(features, target, test_size=0.1, random_state=365)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=365)

print("len x train: ", len(X_train))
print("len x test: ",len(X_test))
print("len y train: ",len(y_train))
print("len y test: ",len(y_test))

len x train:  24105
len x test:  2976
len y train:  24105
len y test:  2976


In [7]:
X_train.shape

(24105, 25)

In [8]:
X_train_array = X_train.to_numpy()
y_train_array = y_train.to_numpy()

In [9]:
print(X_train_array.shape)
print(y_train_array.shape)

(24105, 25)
(24105,)


In [10]:
X_train_array = X_train_array.reshape((X_train.shape[0],SIZE,SIZE,1))
y_train_array = y_train_array.reshape((y_train.shape[0],1))

In [11]:
print(X_train_array.shape)
print(y_train_array.shape)

(24105, 5, 5, 1)
(24105, 1)


In [12]:
X_val_array = X_val.to_numpy()
X_val_array = X_val_array.reshape((X_val.shape[0],SIZE,SIZE,1))
y_val_array = y_val.to_numpy()
y_val_array = y_val_array.reshape((y_val.shape[0],1))

X_test_array = X_test.to_numpy()
X_test_array = X_test_array.reshape((X_test.shape[0],SIZE,SIZE,1))
y_test_array = y_test.to_numpy()
y_test_array = y_test_array.reshape((y_test.shape[0],1))

In [13]:
import tensorflow as tf

input_shape = (SIZE, SIZE, 1)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),

    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(), 

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 5, 5, 32)       │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 2, 2, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 2, 2, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 1, 1, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,041 (90.00 KB)

 Trainable params: 23,041 (90.00 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [15]:
history = model.fit(X_train_array, y_train_array, 
    validation_data=(X_val_array, y_val_array),
    epochs=10,
    batch_size=128)

Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.6490 - loss: 0.6419 - val_accuracy: 0.6827 - val_loss: 0.6075
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.6825 - loss: 0.6019 - val_accuracy: 0.6879 - val_loss: 0.5898
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.6859 - loss: 0.5899 - val_accuracy: 0.6913 - val_loss: 0.5861
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6914 - loss: 0.5874 - val_accuracy: 0.6991 - val_loss: 0.5831
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.6943 - loss: 0.5802 - val_accuracy: 0.6876 - val_loss: 0.5987
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.6968 - loss: 0.5772 - val_accuracy: 0.6973 - val_loss: 0.5819
Epoch 7/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.7007 - loss: 0.5729 - val_accuracy: 0.6898 - val_loss: 0.5798
Epoch 8/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.7041 - loss: 0.5657 - val_acc

In [16]:
test_loss, test_acc = model.evaluate(X_test_array, y_test_array)
print(f"Test accuracy: {test_acc:.3f}")

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6746 - loss: 0.5916
Test accuracy: 0.683


In [17]:
evaluate_model(model, X_test_array, y_test_array)

93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

===== Evaluation Results =====
┌──────────────┬────────────┬────────────┐
│              │ Pred=Alive │ Pred=Dead  │
├──────────────┼────────────┼────────────┤
│ True=Alive   │        436 │        617 │
│ True=Dead    │        327 │       1596 │
└──────────────┴────────────┴────────────┘

--- Performance Metrics ---
Accuracy    : 0.683
Precision   : 0.571
Recall      : 0.414
F1-score    : 0.480
